In [79]:
%matplotlib inline

import csv
import os
import numpy as np
import shutil
import skimage.io
import tempfile
import xarray as xr
import matplotlib.pyplot as plt
from slicedimage import ImageFormat
import seaborn as sns
from typing import Iterable, List, Optional, Set, Tuple, Union

import starfish.data
from starfish.types import Axes, Levels
from starfish import FieldOfView
from starfish.image import Filter
from starfish.util.plot import imshow_plane, intensity_histogram
from starfish import DecodedIntensityTable, IntensityTable
from starfish.core.experiment.builder.structured_formatter import format_structured_dataset
from starfish import data, display
from starfish.image import ApplyTransform, LearnTransform
from starfish.spots import FindSpots
from starfish.util.plot import imshow_plane
from starfish.core.spots.DecodeSpots.trace_builders import build_spot_traces_exact_match
from starfish.spots import DecodeSpots
from starfish.types import TraceBuildingStrategies
from starfish.core.types import ArrayLike, Axes, Coordinates, Number
from starfish.morphology import BinaryMaskCollection
from starfish.core.morphology.label_image import LabelImage
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.core.intensity_table.intensity_table import IntensityTable

# Loading images

In [7]:
inputdir = "D:/Tatsuya/STARmap/220503_5d_tip_dev_markers_benchmark/FOV1/starfish_input/"
primary_dir =  "D:/Tatsuya/STARmap/220503_5d_tip_dev_markers_benchmark/FOV1/"
primary_out = os.path.join(primary_dir, "data_out")

In [4]:
#YOU NEED TO RUN THIS ONLY ONCE!!

#Convert structured data into SpaceTx Format
#dimentin of coordinates.csv and image files in the directory should match

primary_out = os.path.join(primary_dir, "data_out")
os.makedirs(primary_out, exist_ok=True)


format_structured_dataset(
    inputdir,
    os.path.join(inputdir, "coordinates.csv"),
    primary_out,
    ImageFormat.TIFF,
)


In [ ]:
# Don’t forget to replace the fake codebook.json in the output directory
with open(os.path.join(primary_out, "codebook.json"), "r") as fh:
    print(fh.read())

In [ ]:
#Load up the experiment¶

from starfish import Experiment

exp = Experiment.from_json(os.path.join(primary_out, "experiment.json"))

print(exp.fovs())
exp

# Image clipping and filtering

In [89]:
#Trim empty space
fov_xmin = 170
fov_xmax = 400
fov_ymin = 0
fov_ymax = 1428

#loading fov and image
fov = exp['fov_000']
image = fov.get_image("primary", x=slice(fov_xmin, fov_xmax), y=slice(fov_ymin, fov_ymax)) 
image # non-matched z steps between imaging rounds causes error.
#unfiltered image
image_orig = image

<starfish.ImageStack (r: 7, c: 4, z: 202, y: 1428, x: 230)>

In [85]:
##==##==##==##==##==##==##==##==##==##==
##==##==##==##==##==##==##==##==##==##==
##crop area####
xmin = 0#400
xmax = 230#700
# ymin = 1000
# ymax = 1800
ymin = 400
ymax = 1400#1428
##==##==##==##==##==##==##==##==##==##==
##==##==##==##==##==##==##==##==##==##==

# crop imagestacks
image = image_orig
crop_selection = {Axes.X: (xmin, xmax), Axes.Y: (ymin, ymax)}
cropped_imgs_orig: image= image.sel(crop_selection)


In [34]:
# bandpass filter to remove cellular background and camera noise
bandpass = starfish.image.Filter.Bandpass(lshort=.5, llong=11, threshold=0.0)#llong=11 seemed to be good

# gaussian blur to smooth z-axis
glp = starfish.image.Filter.GaussianLowPass(
    sigma=(1, 0, 0),
    is_volume=True
)

# clipping
clipper = Filter.Clip(p_min=75, is_volume=True) 

# apply filters
image_filtered: image = bandpass.run(scaled)
image_filtered: image = glp.run(image_filtered)
image_filtered: image = clipper.run(image_filtered)
    
# combine all imaging rounds and channels
cropped_dots: image = image_filtered.reduce({Axes.ROUND, Axes.CH}, func="max")
cropped_dots_orig: image = cropped_imgs_orig.reduce({Axes.ROUND, Axes.CH}, func="max")

# Spot detection

In [44]:

bd = FindSpots.BlobDetector(
    min_sigma=1,
    max_sigma=4,
    num_sigma=10, 
    threshold=0.1,
    exclude_border=False, 
    measurement_type='mean'
)

# find spots on cropped images
bd_spots_ref = bd.run(image_stack=image_filtered, reference_image=cropped_dots)
bd_spots = bd.run(image_stack=image_filtered)

# build spot traces into intensity table
bd_table = build_spot_traces_exact_match(bd_spots_ref)

# spot finding from unfiltered images 
bd_spots_ref_orig = bd.run(image_stack=cropped_imgs_orig, reference_image=cropped_dots_orig)
bd_table_orig = build_spot_traces_exact_match(bd_spots_ref_orig)

# create spot table
spot_table = bd_table.to_features_dataframe()
spot_table

In [ ]:
# decode the pixel traces using the codebook
codebook=exp.codebook
decoder = DecodeSpots.SimpleLookupDecoder(codebook=codebook)#perRoundMax did not work in this test...
decoded_bd = decoder.run(spots=bd_spots)
data = decoded_bd.to_features_dataframe()
data

# Single-cell analysis

In [91]:
##generating segmentation mask
##running what "from_external_labeled_imag" does. this function does not consider ZPLANE, so I added this feature.

#import segmentation mask from PlantSeg
#converted .h5 file from PlantSeg to Tiff on ImageJ
seg_path='D:/Tatsuya/STARmap/220503_5d_tip_dev_markers_benchmark/FOV1/segmentation/PreProcessing_patch40-80-80/confocal_unet_bce_dice_ds3x/MultiCut_07_03_1_0_1_1/R1_4th_from_right_R1_FOV1_Merged_RAW_ch02_predictions_multicut.tiff'
label_image = io.imread(seg_path)

#trimming segmentation mask to the same dimensions as spot images
spot_xmin = fov_xmin + xmin
spot_xmax = spot_xmin + xmax - xmin
spot_ymin = fov_ymin + ymin
spot_ymax = spot_ymin + ymax - ymin
label_image = label_image[:, spot_ymin:spot_ymax, spot_xmin:spot_xmax] #[z, y, x]

#trimming original image
image_orig = fov.get_image("primary", x=slice(fov_xmin, fov_xmax), y=slice(fov_ymin, fov_ymax))
crop_selection = {Axes.X: (xmin, xmax), Axes.Y: (ymin, ymax)}
image2: image= image_orig.sel(crop_selection)

100%|█████████████████████████████████████████████████████████████████████████████| 5656/5656 [00:29<00:00, 188.58it/s]


In [92]:
original_image =image2
physical_ticks = {Coordinates.Y: original_image.xarray.yc.values,
                        Coordinates.X: original_image.xarray.xc.values,
                 Coordinates.Z: original_image.xarray.zc.values} #added Z coordinates as this was missing in the original function

# Get the pixel values from the original image
pixel_coords = {Axes.Y: original_image.xarray.y.values,
                        Axes.X: original_image.xarray.x.values}

# Create the label image
label_im = LabelImage.from_label_array_and_ticks(
            label_image,
            pixel_ticks=pixel_coords,
            physical_ticks=physical_ticks,
            log=original_image.log
        )
masks = BinaryMaskCollection.from_label_image(label_im)

In [94]:

# assign spots to cells
ta = AssignTargets.Label()
assigned = ta.run(masks, decoded_bd)


In [ ]:
#create cell by gene matrix
cg = assigned.to_expression_matrix()

#removing background 'cell'
cg2 = cg[cg['cell_id']!='0000']

#cell filtering by total count and normalize data
cg2_filtered = cg2[cg2.data.T.sum(axis=0) > 5, :]
data_norm_filtered = cg2_filtered.data.T / cg2_filtered.data.T.sum(axis=0)

#save matrix with cell id
out = cg2_filtered.to_pandas()
keys = range(0, len(out))
rename = dict(zip(keys,cg2_filtered.cell_id.values ))
out = out.rename(index= rename)
out.to_csv(primary_dir +'_analysis_scale_filter/matrix_filter2.csv')
out

# visualization 

In [100]:
#extract info of individual targets
#use this function to get input data for napari visualization 
def _spots_to_markers_gene_select(intensity_table: IntensityTable, target) -> Tuple[np.ndarray, np.ndarray]:
    data_table = intensity_table.to_features_dataframe()

    #select target gene
    data = data_table[data_table['target']==target]
    
    n_rounds = intensity_table.sizes[Axes.ROUND.value]
    n_channels = intensity_table.sizes[Axes.CH.value]
    n_features = data.shape[0]
    code_length = n_rounds * n_channels
    # create 5-d coordinates for plotting in (x, y, round. channel, z)
    n_markers = n_features*n_rounds*n_channels
    coords = np.zeros((n_markers, 5), dtype=np.uint16)

    # create the coordinates.
    # X, Y, and Z are repeated once per (r, ch) pair (code_length).
    # the cartesian product of (r, c) are created, and tiled once per feature (n_features)
    # we ensure that (ch, round) cycle in c-order to match the order of the linearized
    # array, used below for masking.
    coords[:, 0] = np.tile(np.tile(range(n_rounds), n_channels), n_features)
    coords[:, 1] = np.tile(np.repeat(range(n_channels), n_rounds), n_features)
    coords[:, 2] = np.repeat(data['z'].values, code_length)
    coords[:, 3] = np.repeat(data['y'].values, code_length)
    coords[:, 4] = np.repeat(data['x'].values, code_length)

    sizes = np.repeat(data['radius'].values, code_length)
    rc = np.zeros((sizes.shape[0], 2), dtype=int)
    z = sizes[:, np.newaxis]
    yx = np.tile(sizes[:, np.newaxis], (1, 2))
    sizes = np.concatenate((rc, yx, z), axis=1)
    return coords, sizes

In [102]:
gene_list = ['AT3G46280', 'AT2G40260', 'AT1G71930', 'AT5G64620', 'AT1G16390', 'AT2G46570', 'AT3G54220', 
             'AT1G07640', 'AT5G53730', 'AT5G12050', 'AT1G07710', 'AT2G31310', 'AT4G29100', 'AT5G37800', 
             'AT4G22160', 'AT4G28100', 'AT3G20840', 'AT1G71692', 'AT5G58010', 'AT5G57620', 'AT3G55550',
             'AT5G42630', 'AT2G34140', 'AT3G10080', 'AT5G48657', 'AT1G79840', 'AT4G30080', 'AT2G28900']

In [ ]:
#gene visualization
from matplotlib.pyplot import cm
%gui qt
viewer = display(stack=cropped_imgs_orig, masks=masks)
color = cm.rainbow(np.linspace(0, 1, len(gene_list)))

for i in range(len(gene_list)):
    coords1, sizes1 = _spots_to_markers_gene_select(decoded_bd, gene_list[i])
    viewer.add_points(coords1, face_color=color[i], edge_color =color[i], size=5, symbol="ring", n_dimensional=True, name=gene_list[i])